# Testing Interpolators

In [2]:
import numpy as np
import xarray as xr
import time

from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator


In [3]:
# Define the size of the array
array_size = 100000  # Change this to the desired size of your array

# Generate a random array of integers in the range [0, 30]
x = np.random.randint(0, 31, size=array_size)
y = np.random.randint(0, 31, size=array_size)
data = np.random.uniform(-1, 1, size=array_size)

In [4]:
# Create datasets of 20 elements each
dataset_size = array_size // 20
datasets = []

for i in range(dataset_size):
    start_idx = i * 20
    end_idx = (i + 1) * 20
    x_chunk = x[start_idx:end_idx]
    y_chunk = y[start_idx:end_idx]
    data_chunk = data[start_idx:end_idx]
    
    # Sort the data based on x-coordinates
    sorted_indices = np.argsort(x_chunk)
    x_chunk = x_chunk[sorted_indices]
    y_chunk = y_chunk[sorted_indices]
    data_chunk = data_chunk[sorted_indices]
    
    dataset = {
        'x': x_chunk,
        'y': y_chunk,
        'data': data_chunk
    }
    
    datasets.append(dataset)

In [5]:
# Define interpolation methods (including 'metpy')
interpolation_methods = ['linear', 'nearest']

# Create a grid of coordinates
x_grid, y_grid = np.meshgrid(np.linspace(0, 30, 100), np.linspace(0, 30, 100))
grid_points = np.column_stack((x_grid.flatten(), y_grid.flatten()))

# Initialize lists to store timing results
timing_results = {method: [] for method in interpolation_methods}

# Test the speed of 2D interpolation for various methods and collect timing results
for dataset in datasets:
    x_chunk = dataset['x']
    y_chunk = dataset['y']
    data_chunk = dataset['data']
    
    for method in interpolation_methods:
        start_time = time.time()
        
        if method == 'linear':
            # Use griddata for linear interpolation
            interpolated_data = griddata((x_chunk, y_chunk), data_chunk, grid_points, method='linear')
        elif method == 'nearest':
            # Use griddata for nearest-neighbor interpolation
            interpolated_data = griddata((x_chunk, y_chunk), data_chunk, grid_points, method='nearest')
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        timing_results[method].append(elapsed_time)

# Create a dictionary to store the results
results = {}

# Calculate and print median and maximum times for each interpolation method
for method, times in timing_results.items():
    median_time = np.median(times)
    max_time = np.max(times)
    
    # Store the results in the dictionary
    results[method] = {
        'Median Time': median_time,
        'Maximum Time': max_time
    }
    print(f"2D Interpolation method '{method}':")
    print(f"Median time = {median_time:.4f} seconds")
    print(f"Maximum time = {max_time:.4f} seconds")

2D Interpolation method 'linear':
Median time = 0.0003 seconds
Maximum time = 0.0011 seconds
2D Interpolation method 'nearest':
Median time = 0.0023 seconds
Maximum time = 0.0059 seconds


In [6]:
# Define the two values you want to compare
value1 = results['linear']['Median Time']
value2 = results['nearest']['Median Time']

# Calculate the difference percentage
difference_percentage = ((value2 - value1) / abs(value1)) * 100

# Format the percentage with 2 decimal places
formatted_percentage = f"{difference_percentage:.1f}%"

# Print the formatted percentage
print(f"Linear to Nearest time increase: {formatted_percentage}")

Linear to Nearest time increase: 755.9%
